In [2]:
import pandas as pd
import numpy as np

def analyze_tight_ends(csv_file):
    """
    Analyzes tight end performance and calculates a combined impact score similar to the RB analysis.
    """
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Filter for tight ends only
    te_df = df[df['Pos'] == 'TE'].copy()

    # Calculate key metrics
    te_df['Yards_per_Game'] = te_df['Yds'] / te_df['G']
    te_df['Success_Rate'] = te_df['1D'] / te_df['Rec']  # Using first downs per reception as success rate
    te_df['Yards_per_Reception'] = te_df['Yds'] / te_df['Rec']
    te_df['TD_per_Reception'] = te_df['BrkTkl'] / te_df['Rec']  # Using broken tackles as a proxy for TDs since TDs aren't in dataset

    # Normalize metrics to 0-1 scale
    def normalize(series):
        min_val = series.min()
        max_val = series.max()
        return (series - min_val) / (max_val - min_val) if max_val > min_val else series

    # Normalize each metric
    metrics = ['Yards_per_Reception', 'Success_Rate', 'Yards_per_Game', 'TD_per_Reception']
    for metric in metrics:
        te_df[f'{metric}_norm'] = normalize(te_df[metric])

    # Calculate impact score (weighted average of normalized metrics)
    weights = {
        'Yards_per_Reception_norm': 0.3,
        'Success_Rate_norm': 0.3,
        'Yards_per_Game_norm': 0.2,
        'TD_per_Reception_norm': 0.2
    }

    te_df['Impact_Score'] = sum(te_df[metric] * weight for metric, weight in weights.items())

    # Create final dataframe with desired columns
    result_df = pd.DataFrame({
        'Tm': te_df['Team'],
        'Player': te_df['Player'],
        'Rec': te_df['Rec'],
        'Impact_Score': te_df['Impact_Score'],
        'Yards_per_Reception': te_df['Yards_per_Reception'],
        'Touchdowns_per_Reception': te_df['TD_per_Reception'],
        'Success_Rate': te_df['Success_Rate'],
        'Y/G': te_df['Yards_per_Game']
    })

    # Sort by Impact Score
    result_df = result_df.sort_values('Impact_Score', ascending=False)

    return result_df

# Example usage
if __name__ == "__main__":
    result_df = analyze_tight_ends('football_stats.csv')
    print("\nTop Impactful Tight Ends per Team in 2023 based on Combined Impact Score:")
    print(result_df.to_string(index=True, float_format=lambda x: '{:.6f}'.format(x)))


Top Impactful Tight Ends per Team in 2023 based on Combined Impact Score:
      Tm              Player  Rec  Impact_Score  Yards_per_Reception  Touchdowns_per_Reception  Success_Rate       Y/G
421  HOU   Teagan Quitoriano    2      0.758754            16.500000                  0.500000      1.000000  4.714286
46   SFO       George Kittle   65      0.649057            15.692308                  0.076923      0.646154 63.750000
456  MIN           Nick Muse    1      0.638462            22.000000                  0.000000      1.000000 11.000000
475  TEN        Trevon Wesco    1      0.598948            21.000000                  0.000000      1.000000  1.400000
306  IND     Andrew Ogletree    9      0.590308            16.333333                  0.111111      0.888889 12.250000
429  NYJ        Kenny Yeboah    2      0.583596            14.000000                  0.500000      0.500000  5.600000
92   BAL        Mark Andrews   45      0.572296            12.088889                  0.0888

In [3]:
import pandas as pd
import numpy as np

def analyze_top_tight_ends(csv_file):
    """
    Analyzes and returns the top 2 tight ends per team based on impact score.
    """
    # Read the CSV file
    df = pd.read_csv(csv_file)

    # Filter for tight ends only
    te_df = df[df['Pos'] == 'TE'].copy()

    # Calculate key metrics
    te_df['Yards_per_Game'] = te_df['Yds'] / te_df['G']
    te_df['Success_Rate'] = te_df['1D'] / te_df['Rec']
    te_df['Yards_per_Reception'] = te_df['Yds'] / te_df['Rec']

    # Normalize metrics to 0-1 scale
    def normalize(series):
        min_val = series.min()
        max_val = series.max()
        return (series - min_val) / (max_val - min_val) if max_val > min_val else series

    metrics = {
        'Rec': 0.2,           # Volume of receptions
        'Yards_per_Game': 0.3,     # Production
        'Success_Rate': 0.3,       # Efficiency
        'BrkTkl': 0.2             # Playmaking ability
    }

    # Calculate normalized scores
    impact_score = 0
    for metric, weight in metrics.items():
        te_df[f'{metric}_norm'] = normalize(te_df[metric])
        impact_score += te_df[f'{metric}_norm'] * weight

    te_df['impact_score'] = impact_score

    # Get top 2 TEs per team
    top_tes = (te_df.sort_values('impact_score', ascending=False)
               .groupby('Team')
               .agg({
                   'Player': lambda x: list(x)[:2],
                   'impact_score': lambda x: list(x)[:2],
                   'Rec': lambda x: list(x)[:2],
                   'Yds': lambda x: list(x)[:2]
               }))

    return top_tes

def print_top_tes(top_tes):
    """Pretty prints the top 2 TEs per team"""
    print("\nTop 2 Tight Ends per Team")
    print("-" * 80)
    print(f"{'Team':<8} {'TE1':<25} {'Impact Score':<12} {'TE2':<25} {'Impact Score':<12}")
    print("-" * 80)

    for team, row in top_tes.iterrows():
        te1_name = row['Player'][0] if len(row['Player']) > 0 else "N/A"
        te1_score = f"{row['impact_score'][0]:.3f}" if len(row['impact_score']) > 0 else "N/A"
        te2_name = row['Player'][1] if len(row['Player']) > 1 else "N/A"
        te2_score = f"{row['impact_score'][1]:.3f}" if len(row['impact_score']) > 1 else "N/A"

        print(f"{team:<8} {te1_name:<25} {te1_score:<12} {te2_name:<25} {te2_score:<12}")

# Run the analysis
if __name__ == "__main__":
    top_tes = analyze_top_tight_ends('football_stats.csv')
    print_top_tes(top_tes)

    # Print additional stats for verification
    print("\nAdditional Stats for Verification:")
    print(f"Total teams analyzed: {len(top_tes)}")


Top 2 Tight Ends per Team
--------------------------------------------------------------------------------
Team     TE1                       Impact Score TE2                       Impact Score
--------------------------------------------------------------------------------
ARI      Trey McBride              0.722        Elijah Higgins            0.292       
ATL      Jonnu Smith               0.525        Kyle Pitts                0.474       
BAL      Mark Andrews              0.589        Isaiah Likely             0.439       
BUF      Dalton Kincaid            0.503        Dawson Knox               0.314       
CAR      Tommy Tremble             0.293        Hayden Hurst              0.268       
CHI      Cole Kmet                 0.553        Robert Tonyan             0.241       
CIN      Tanner Hudson             0.409        Drew Sample               0.233       
CLE      David Njoku               0.725        Jordan Akins              0.250       
DAL      Jake Ferguson      